In [1]:
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import make_friedman2, make_regression
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelBinarizer, OneHotEncoder
from sklearn.linear_model import LinearRegression, SGDRegressor, ElasticNet, RidgeCV
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, GradientBoostingRegressor,VotingRegressor, StackingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_squared_log_error, mean_absolute_error
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.svm import LinearSVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline, make_union
from sklearn.base import BaseEstimator, TransformerMixin

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [2]:
pd.options.display.max_columns = None

cv = KFold(n_splits=5)
simple_pre_flag = True
scale_data = False

cat_cols = ['Neutered/Spayed', 'Gender', 'MIXED_BREED_FLAG', 'BREED', 'neuter_dt']
num_cols = ['ALB', 'ALKP', 'ALT', 'AMYL', 'BUN', 'CA', 'CHOL', 'CREA', 'EOSINOPHIL', 'GLOB', 'GLU',
       'HCT', 'HGB', 'LYMPHOCYTE', 'MCH', 'MCHC', 'MCV', 'MONOCYTE', 'MPV',
       'PHOS', 'PLT', 'RBC', 'RDW', 'TBIL', 'TP', 'WBC']

In [3]:
if simple_pre_flag:
    X_train = pd.read_csv('data/X_train_pre_simple.csv')
    X_test = pd.read_csv('data/X_test_pre_simple.csv')
    y_train = pd.read_csv('data/y_train_pre_simple.csv').to_numpy()
    y_test = pd.read_csv('data/y_test_pre_simple.csv').to_numpy()

else:
    X_train = pd.read_csv('data/X_train_pre.csv')
    X_test = pd.read_csv('data/X_test_pre.csv')
    y_train = pd.read_csv('data/y_train_pre.csv').to_numpy()
    y_test = pd.read_csv('data/y_test_pre.csv').to_numpy()

X_train.head()

,MIXED_BREED_FLAG_0,MIXED_BREED_FLAG_1,BREED_0,BREED_1,BREED_2,BREED_3,BREED_4,BREED_5,BREED_6,BREED_7,BREED_8,BREED_9,BREED_10,BREED_11,BREED_12,BREED_13,BREED_14,BREED_15,BREED_16,BREED_17,BREED_18,BREED_19,BREED_20,BREED_21,BREED_22,BREED_23,BREED_24,BREED_25,BREED_26,BREED_27,BREED_28,BREED_29,BREED_30,BREED_31,BREED_32,BREED_33,BREED_34,BREED_35,BREED_36,BREED_37,BREED_38,BREED_39,BREED_40,BREED_41,BREED_42,BREED_43,BREED_44,BREED_45,BREED_46,BREED_47,BREED_48,BREED_49,BREED_50,BREED_51,BREED_52,BREED_53,BREED_54,BREED_55,BREED_56,BREED_57,BREED_58,BREED_59,BREED_60,BREED_61,BREED_62,BREED_63,BREED_64,BREED_65,BREED_66,BREED_67,BREED_68,BREED_69,BREED_70,BREED_71,BREED_72,BREED_73,BREED_74,BREED_75,BREED_76,BREED_77,BREED_78,BREED_79,BREED_80,BREED_81,BREED_82,BREED_83,BREED_84,BREED_85,BREED_86,BREED_87,BREED_88,BREED_89,BREED_90,BREED_91,BREED_92,BREED_93,BREED_94,neuter_dt_0,ALB,ALKP,ALT,AMYL,BUN,CA,CHOL,CREA,EOSINOPHIL,GLOB,GLU,HCT,HGB,LYMPHOCYTE,MCH,MCHC,MCV,MONOCYTE,MPV,PHOS,PLT,RBC,RDW,TBIL,TP,WBC,Neutered/Spayed_0,Neutered/Spayed_1,Neutered/Spayed_2,Gender_0
0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.565117,-0.083936,3.298568,0.055539,-0.923631,1.045984,0.639321,-0.119074,3.928146e-16,1.405175,0.291423,0.981101,7.352667e-01,-1.405885e-15,6.288612e-02,-4.646169e-02,4.715268e-01,-1.400820,-4.903108e-01,-0.593610,-1.944974e+00,0.079278,-2.674955e-01,0.801551,1.394026,1.218854e-01,1.0,0.0,0.0,1.0
1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.014025,3.139680,0.555261,-0.959741,-0.690719,0.441009,-1.186900,-1.000346,3.928146e-16,-1.126764,-0.937075,0.000000,8.597155e-16,-1.405885e-15,4.628292e-16,1.066452e-15,1.727638e-15,0.000000,1.207058e-15,1.676055,1.485744e-15,0.000000,2.121587e-15,-0.530343,-1.410314,-4.822035e-17,1.0,0.0,0.0,1.0
2,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,2.144259,-0.517364,-0.391833,-0.954587,-1.389455,0.138522,1.210015,-0.119074,-9.681104e-01,-0.765058,0.099470,-0.146499,-2.447860e-01,2.259567e+00,4.725315e-02,-9.599132e-02,4.715268e-01,-0.535286,-5.854428e-01,-0.671874,-1.396087e+00,-0.091589,-3.112883e-01,0.040468,0.552723,-1.778024e-01,1.0,0.0,0.0,1.0
3,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.565117,-0.652810,-1.077660,3.802281,-0.457807,-0.466454,-1.985872,0.006822,-9.681104e-01,0.320059,-0.246045,-0.941572,-7.075889e-01,7.855043e-01,-2.244386e-02,1.951833e-01,-3.794750e-01,1.368889,-1.641447e-01,-0.906667,1.485744e-15,-0.133847,3.296782e-01,0.230739,0.412507,-8.985056e-02,0.0,1.0,0.0,-1.0
4,-1.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [4]:
if scale_data:
    scaler = StandardScaler() #StandardScaler() #RobustScaler() # or MinMaxScaler()

    X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
    X_test[num_cols] = scaler.transform(X_test[num_cols])

    #X_train = scaler.fit_transform(X_train)
    #X_test = scaler.transform(X_test)

X_train, X_test = X_train.to_numpy(), X_test.to_numpy()
X_train.shape, y_train.shape

((721512, 128), (721512, 1))

In [5]:
model = LGBMRegressor(n_jobs=-1)

n_scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
print('MAE: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

MAE: -2.113 (0.004)


In [6]:
model = CatBoostRegressor(thread_count=-1)

n_scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
print('MAE: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

MAE: -2.085 (0.004)


In [7]:
model = XGBRegressor(n_jobs=-1)

n_scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
print('MAE: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

KeyboardInterrupt: 

In [ ]:
model = RandomForestRegressor(random_state=42, n_jobs=-1)
                    
n_scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
print('MAE: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

In [ ]:
# Ensemble regressors
reg1 = GradientBoostingRegressor(random_state=1)
reg2 = RandomForestRegressor(random_state=42, n_jobs=-1) # takes longest
reg3 = SGDRegressor(max_iter=1000, tol=1e-3)
reg4 = XGBRegressor(n_jobs=-1)
reg5 = LGBMRegressor(n_jobs=-1)
reg6 = CatBoostRegressor(thread_count=-1)

In [ ]:
vote_reg = VotingRegressor(estimators=[('gb', reg1), ('rf', reg2), ('sgd', reg3), ('xgbr', reg4), ('lgbmr', reg5), ('cbr', reg6)])

n_scores = cross_val_score(vote_reg, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
print('MAE: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

In [ ]:
stack_reg = StackingRegressor(
    estimators=[('gb', reg1), ('rf', reg2), ('lr', reg3), ('xgbr', reg4), ('lgbmr', reg5), ('cbr', reg6)],
    final_estimator=XGBRegressor()
)

n_scores = cross_val_score(stack_reg, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
print('MAE: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

In [ ]:
stack_reg.fit(X_train, y_train)
print('lRMSE: %.3f' % mean_squared_log_error(y_test, stack_reg.predict(X_test)))

In [ ]:
model = CatBoostRegressor(eval_metric='MAE', thread_count=-1, metric_period=100)
model.fit(X_train, y_train)

In [ ]:
print("MAE: ", mean_absolute_error(y_test, model.predict(X_test)))
for i in range(10):
    test_item = X_test[i].reshape(1, -1)
    print("GT: ", y_test[i])
    print("Regressor: ", model.predict(test_item))